In [216]:
from working_with_files import *
from preprocessing import *
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from collections import Counter

In [263]:
def sort_by_official_dates(df):
    df1 = pd.DataFrame() # < 2008
    df2 = pd.DataFrame() # 2008 - 2013
    df3 = pd.DataFrame() # 2013 - 2016
    df4 = pd.DataFrame() # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date <= date1:
                df1 = df1.append(row, ignore_index=True)
            if date > date1 and date <= date2:
                df2 = df2.append(row, ignore_index=True)
            if date > date2 and date <= date3:
                df3 = df3.append(row, ignore_index=True)
            if date > date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def sort_by_unofficial_dates(df):
    df1 = pd.DataFrame() # < 2004
    df2 = pd.DataFrame() # 2004 - 2008
    df3 = pd.DataFrame() # 2008 - 2014
    df4 = pd.DataFrame() # > 2014
    date1 = datetime.strptime('01-01-2004', '%d-%m-%Y').date()
    date2 = datetime.strptime('01-01-2008', '%d-%m-%Y').date()
    date3 = datetime.strptime('01-01-2014', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date < date1:
                df1 = df1.append(row, ignore_index=True)
            if date >= date1 and date < date2:
                df2 = df2.append(row, ignore_index=True)
            if date >= date2 and date < date3:
                df3 = df3.append(row, ignore_index=True)
            if date >= date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def get_stat(dfs):
    n = 0
    for df in dfs:
        print('Corpus {}:'.format(n), len(df), 'texts', len(get_all_texts(df).split()), 'words')
        n += 1

def show_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic {}:'.format(topic_idx))
        print(' '.join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()

def get_topics(df, n_topics, n_top_words, models, stop_topics):
    
    documents = df.loc[:, 'text_clean']
    
    random_seed = 42
    
    if 'nmf' in models:
        tfidf = tfidf_vectorizer.fit_transform(documents)
        tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        
        nmf = NMF(n_components=n_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
        
        print('NMF')
        show_topics(nmf, tfidf_feature_names, n_top_words)
        
        doc_topic = nmf.transform(tfidf)
        
    if 'lda' in models:
        tf = tf_vectorizer.fit_transform(documents)
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)
        
        print('LDA')
        show_topics(lda, tf_feature_names, n_top_words)
        
        doc_topic = lda.transform(tf)
    
    topics = []    
    for n in range(doc_topic.shape[0]):
        top_topics = doc_topic[n].argsort()[-3:][::-1] # topics numbers
        top_topics_scores = doc_topic[n][top_topics]
        print(top_topics)
        print(top_topics_scores)
        i = 0
        try:
            while top_topics[i] in stop_topics:
                #top_topic = top_topics[1]
                i += 1
        except IndexError:
            i = 0
        if top_topics_scores[i] == 0:
            i = 0
        top_topic = top_topics[i]
        topics.append(top_topic)
    df['topic_number'] = topics
        
    return df

### Sorting data by official dates

In [218]:
# read df and split it into 4 separate df4 based on publication date
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs = sort_by_official_dates(df)

In [49]:
# statistics 
get_stat(sorted_dfs)

Corpus 0: 1583 texts 845988 words
Corpus 1: 3168 texts 1111520 words
Corpus 2: 3344 texts 1082001 words
Corpus 3: 4277 texts 1420393 words


In [219]:
# get unnecessary symbols to remove them from texts
symbols_to_del = generate_symbols_to_del(get_all_texts(df))
symbols_to_del.extend(list('abcdefghijklmnopqrstuvwxyz'))
symbols_to_del_upd = [symbol for symbol in symbols_to_del if symbol not in '0123456789']
print(symbols_to_del_upd)

# clean texts from unnecessary symbols
for df in sorted_dfs:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del_upd,))

['=', '%', '‑', '«', '`', '+', '„', '№', '[', ';', '–', '…', ':', '́', '(', '!', '&', '€', ',', '#', ']', '?', '’', '@', '<', '！', '\n', '−', '"', '.', '·', '{', '‒', '´', '―', '$', '>', '—', '°', '－', '\xad', '_', '*', '/', '“', ')', '»', '”', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [220]:
# get and preprocess stopwords
# source: ruscorpora.ru

stopwords_raw = get_lexicon_from_file('russian_stopwords.txt')
stopwords = []
stopwords_string = ''.join(stopwords_raw)
strings = stopwords_string.split('\t')
for string in strings:
    if string.isalpha():
        stopwords.append(string)

# lemmatize stopwords

stopwords_lemmatized = [lemmatize_pymorphy(word) for word in stopwords]
stopwords_lemmatized = list(set(stopwords_lemmatized))
stopwords = [stopword for stopword in stopwords_lemmatized if stopword.isalpha()]

stopwords.extend('''б г д е ж з й л м н п р т ф х ц ч ш щ ъ ы ь э ю
                наш твой мой свой ваш его их ее над чей такой другой между год страна государство также все отношение г-н 
                россия российский сторона де два дж мид внешнеполитический интервью министр газета иностранный дело
                катерина спецкор новость рассказывать подобный кроме министерство кстати пресс-конференция москва сайт лавров
                медведев путин сайт подробный рф s форум сердечно добрый желать приветствовать сам ясный ясность кто-то
                говорить хотеть официальный еще какой-то об лишь включая вокруг ле никакой во ясно ярмарка'''.split())

# print(stopwords)

In [221]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b\d*['а-я-]+\b")
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b\d*['а-я-]+\b")

In [264]:
df1 = get_topics(sorted_dfs[0], 35, 10, ['nmf'], [0, 2]) # до 2008

NMF
Topic 0:
международный развитие политика новый мир мировой интерес решение проблема становиться
Topic 1:
израиль палестинец дорожный карта израильский палестино-израильский палестинский четверка посредник квартет
Topic 2:
сотрудничество вопрос обсуждать переговоры визит договариваться президент двусторонний встреча договоренность
Topic 3:
палестинский пна аббас хамас администрация территория межпалестинский глава правительство национальный
Topic 4:
европа совет се европейский председательство конвенция комитет парламентский правовой право
Topic 5:
исламский цивилизация оик религия ислам религиозный православный мусульманский мир церковь
Topic 6:
соотечественник рубеж диаспора конгресс русскоязычный русский правительственный конференция родина работа
Topic 7:
грузинский грузия конфликт тбилиси абхазия зона осетия миротворец южный грузино-осетинский
Topic 8:
ирак иракский багдад иракец восстановление национальный правительство выборы оон суверенитет
Topic 9:
обсе организация бдипча с

[0.25791326 0.0222031  0.00714805]
[ 3 10  1]
[0.25362587 0.12260538 0.02599438]
[10  5  0]
[0.13017342 0.03245222 0.01648431]
[15  0 12]
[0.19567287 0.03437223 0.02992809]
[ 2  0 29]
[0.04153222 0.03694694 0.01576114]
[10 16  0]
[0.28900395 0.04616377 0.01864742]
[ 6 10  0]
[0.04329846 0.02454377 0.01320179]
[ 7  0 27]
[0.33347656 0.03521883 0.01404025]
[22  0  2]
[0.07758107 0.02218697 0.00812522]
[22 11  0]
[0.13135949 0.05054123 0.03589825]
[26  0 20]
[0.17394079 0.04815416 0.02861753]
[27 10  0]
[0.13584615 0.10284681 0.00680596]
[1 3 0]
[0.27246674 0.02265928 0.01092774]
[10  0  2]
[0.0402847  0.01705434 0.00797516]
[10  8  0]
[0.18202842 0.12154943 0.0057953 ]
[10  5  2]
[0.21432223 0.04107397 0.00798539]
[ 1 10  3]
[0.19309945 0.15781538 0.07557948]
[16 27 10]
[0.1347528  0.12173981 0.06398989]
[10  2  0]
[0.15556188 0.00324711 0.00226059]
[ 5  0 16]
[0.37660267 0.05989977 0.01837679]
[20  1 10]
[0.15487234 0.0677955  0.04165288]
[10 29 34]
[0.27524918 0.01642472 0.        ]
[2

[0.14579737 0.13648029 0.11629575]
[30  0 27]
[0.20281767 0.07818418 0.06724972]
[13  0 29]
[0.41490335 0.08596331 0.02843864]
[12  0  4]
[0.26749099 0.15092492 0.05083279]
[ 5 30  0]
[0.08107279 0.07927363 0.07811323]
[27 17  8]
[0.2264995  0.05133053 0.02990692]
[ 0 27 14]
[0.09267619 0.07835736 0.03469566]
[ 6 28  0]
[0.26868459 0.19786556 0.07695835]
[18 27  0]
[0.0723594  0.05863964 0.03378106]
[ 9  0 16]
[0.28820211 0.09276253 0.04681742]
[ 0 17 27]
[0.1467966  0.03836988 0.02767295]
[ 0 27 17]
[0.08631057 0.08320954 0.06674301]
[ 0 29  6]
[0.11630558 0.04356374 0.02136107]
[29  0 13]
[0.05629509 0.03795932 0.02911171]
[ 5  0 23]
[0.18175649 0.12446753 0.11410642]
[12  0  4]
[0.21937508 0.16518952 0.04352136]
[ 0 29  2]
[0.07564158 0.05023597 0.00491425]
[ 9  0 29]
[0.23033521 0.0750264  0.02295629]
[ 0  9 29]
[0.11757273 0.10568336 0.01534994]
[14  0  2]
[0.67185181 0.03412196 0.00738604]
[ 0 29  6]
[0.11629814 0.05069932 0.02177325]
[ 4  0 16]
[0.28036756 0.11939546 0.07767902]

[0.31884484 0.04663223 0.0416006 ]
[ 5 28 33]
[0.15781983 0.0794051  0.05893099]
[29 15 24]
[0.23056108 0.06229403 0.03856191]
[2 0 4]
[0.08724958 0.03411639 0.02725187]
[ 0 16 24]
[0.06797719 0.03953756 0.03291504]
[ 0  5 28]
[0.12131634 0.035091   0.02265108]
[30 28 15]
[0.06605743 0.03697465 0.03066858]
[ 0 11 22]
[0.08369258 0.06706778 0.05918012]
[15  0  2]
[0.23188318 0.07496916 0.04584627]
[15  2 16]
[0.13387959 0.09536873 0.00848832]
[ 2 27  0]
[0.11212813 0.09380274 0.01659116]
[ 0 32 29]
[0.12017273 0.05788768 0.04690338]
[29  0  2]
[0.13937733 0.04712923 0.0372832 ]
[17  2 16]
[0.28326162 0.05668732 0.02325565]
[28  6  0]
[0.30459439 0.16424729 0.03987621]
[17  0  2]
[0.39006643 0.05403707 0.0490834 ]
[5 2 0]
[0.23162934 0.05878259 0.03169374]
[ 0 22 19]
[0.1789023  0.05902828 0.04541036]
[ 2  0 27]
[0.09347366 0.04836296 0.01335502]
[ 0  8 19]
[0.22993544 0.03249267 0.02965192]
[22  2  0]
[0.1328574  0.11604109 0.06218678]
[30 19  0]
[0.38553624 0.01837192 0.00791107]
[8 0 

[0.0993627  0.04230067 0.02246731]
[32  2 18]
[0.13296038 0.08640985 0.07263909]
[ 0  2 32]
[0.10401034 0.07447877 0.03412484]
[ 2 32  0]
[0.08367097 0.04406498 0.0274471 ]
[ 2  0 32]
[0.08401654 0.07780365 0.02312988]
[19  2 26]
[0.29697354 0.06575474 0.06153255]
[19  0  2]
[0.25916522 0.02843847 0.01426334]
[ 0 19 27]
[0.07777056 0.07317431 0.04181675]
[9 2 1]
[0.23325254 0.06454305 0.03440988]
[9 2 0]
[0.2321251  0.05407823 0.05101421]
[9 0 2]
[0.08527498 0.05541119 0.04487711]
[9 0 4]
[0.28963269 0.09378472 0.04016024]
[9 2 0]
[0.1073818  0.08487928 0.03399832]
[ 0 24 16]
[0.25235347 0.05624379 0.04381324]
[ 2 34  8]
[0.10081328 0.         0.        ]
[33  0 25]
[0.29074103 0.08759237 0.00971875]
[ 2 24  9]
[0.0597517  0.0553874  0.05094525]
[ 2 32 34]
[1.77415147e-01 9.48089691e-05 0.00000000e+00]
[12  0  2]
[0.20946035 0.09767105 0.04214811]
[ 0 22  2]
[0.14757555 0.09100396 0.04765647]
[21  0 29]
[0.33134184 0.05507638 0.01680843]
[ 0  2 24]
[0.08040102 0.03555654 0.02780894]
[ 

In [265]:
nums_rate = dict(sorted(dict(Counter(list(df1.loc[:, 'topic_number']))).items(), key=lambda kv: kv[1], reverse=True))
len(nums_rate)

35

In [266]:
nums_rate

{27: 109,
 29: 87,
 16: 76,
 12: 75,
 30: 73,
 22: 72,
 8: 67,
 1: 66,
 10: 62,
 9: 55,
 7: 54,
 4: 50,
 24: 49,
 11: 47,
 32: 45,
 5: 44,
 21: 44,
 17: 43,
 25: 41,
 28: 40,
 19: 39,
 18: 38,
 15: 37,
 6: 32,
 20: 30,
 3: 28,
 26: 28,
 33: 24,
 23: 24,
 31: 21,
 34: 20,
 13: 20,
 0: 19,
 2: 14,
 14: 10}

In [267]:
df2 = get_topics(sorted_dfs[1], 45, 15, ['nmf'], [0, 2, 11, 35, 41]) # 2008 - 2013

NMF
Topic 0:
международный безопасность мир развитие новый глобальный мировой становиться политика работа решение интерес основа система проблема
Topic 1:
абхазия абхазский закавказье женевский республика дискуссия граница сухум безопасность стабильность женева грузия неприменение инцидент соглашение
Topic 2:
сотрудничество двусторонний взаимодействие сфера обсуждать коллега переговоры область сегодня вопрос торговый-экономический связь договариваться развитие договоренность
Topic 3:
осетия южный югоосетинский цхинвал республика август абхазия сила агрессия грузия территория миротворец неприменение военный граница
Topic 4:
соотечественник рубеж конференция зарубежный конгресс проживать всемирный правительственный община диаспора координационный русский комиссия работа федерация
Topic 5:
сша американский российский-американский вашингтон про обама клинтон госдепартамент президент администрация госсекретарь штат снв вопрос государственный
Topic 6:
одкб коллективный организация безопаснос

[ 2 16  0]
[0.08706449 0.02654351 0.02418276]
[44 31  2]
[0.32005243 0.11971546 0.00132018]
[42 10 31]
[0.32322109 0.07155764 0.02916128]
[36  5 31]
[0.06462379 0.04884026 0.04037265]
[11  2 29]
[0.09420001 0.08118798 0.04786094]
[36  5 31]
[0.1315047  0.05346874 0.02202202]
[31 24 41]
[0.06971187 0.06184472 0.03406554]
[ 7  0 33]
[0.23773059 0.01244518 0.01075509]
[40  0 13]
[0.28662513 0.05314275 0.01737547]
[10 38 31]
[0.10869843 0.04257684 0.01852622]
[42 10 26]
[0.18018569 0.12450212 0.09810924]
[29 35  0]
[0.31532875 0.03900427 0.02568605]
[11 35 29]
[0.0820706  0.06402811 0.06004433]
[33 31  0]
[0.16671843 0.06609454 0.04241898]
[ 5 37  0]
[0.08595443 0.04960965 0.02068383]
[ 9 31 21]
[0.07792086 0.02983993 0.02903431]
[ 5 16 31]
[0.11237445 0.04863479 0.04393281]
[ 5  0 16]
[0.10746247 0.03009942 0.01247048]
[13 31 37]
[0.10743937 0.05452028 0.03023726]
[ 9 35 21]
[0.18122798 0.02216901 0.01354741]
[10 38  0]
[0.25280898 0.04097897 0.02482056]
[10 38  0]
[0.20371154 0.09044657 

[31 35 38]
[0.07340061 0.01103129 0.00485444]
[28 31 36]
[0.27352713 0.03880583 0.03534982]
[28 31 37]
[0.10902524 0.06080477 0.03393679]
[40 11  0]
[0.15086437 0.02234171 0.02130725]
[33 11 41]
[0.11698207 0.10462982 0.09208689]
[10 35 11]
[0.1362785  0.06441342 0.01982264]
[10 31 38]
[0.20748113 0.02070415 0.02044667]
[32 31 38]
[0.16692536 0.06467216 0.03776254]
[ 0 31  4]
[0.06106059 0.04802762 0.01001224]
[11 35  2]
[0.13146584 0.03302587 0.01436419]
[ 2 35 11]
[0.08184439 0.05664809 0.04692984]
[ 6 35  9]
[0.42321737 0.03886736 0.012807  ]
[15  0  2]
[0.2036278  0.04395384 0.02234807]
[10  0 41]
[0.16671456 0.0241093  0.02107756]
[24  0 38]
[0.09675442 0.05081019 0.04945186]
[ 5 28 31]
[0.06479716 0.03125782 0.02827245]
[16 13 12]
[0.0922175  0.0797877  0.06492303]
[26 31  4]
[0.2531895  0.0310397  0.01758549]
[28 31 35]
[0.25320089 0.04961843 0.03512522]
[35  9  0]
[0.10348292 0.06004086 0.03263849]
[15 35  6]
[0.20716945 0.05553816 0.03679184]
[ 7 33  0]
[0.17322072 0.03388027 

[0.16472989 0.06593771 0.06383103]
[30  4 35]
[0.15619141 0.07432933 0.0319263 ]
[38  0 37]
[0.18140589 0.02489314 0.00761197]
[38  0 40]
[0.07739476 0.02369718 0.00343222]
[ 7 35 31]
[0.12399902 0.05502792 0.0183424 ]
[ 7 31  2]
[0.11771691 0.01583863 0.01453377]
[23 19 35]
[0.09572717 0.04830682 0.04048661]
[35  0 30]
[0.06994806 0.05948509 0.02598334]
[26 17 14]
[0.35009156 0.12087707 0.04624816]
[38 41 18]
[0.11669193 0.06248785 0.04329902]
[22  0  2]
[0.14736733 0.03023383 0.02908189]
[38  0 10]
[0.07027108 0.02745447 0.01694784]
[38 31 23]
[0.14100629 0.02879999 0.00749532]
[ 4 30 37]
[0.21212292 0.13552035 0.02223328]
[ 0 38  8]
[0.07499809 0.06185323 0.05173749]
[ 0 29 37]
[0.07641234 0.02098276 0.02020189]
[15 38 35]
[0.18393129 0.05722294 0.05582576]
[11 41  2]
[0.15896028 0.08738938 0.01425111]
[18 31 38]
[0.05079276 0.04763196 0.03902492]
[ 0 38 37]
[0.08515689 0.03905321 0.01622321]
[23  0 14]
[0.06793534 0.05403029 0.0113663 ]
[38 35  0]
[0.12921555 0.06184068 0.06017789]

[0.13806583 0.0604614  0.01649742]
[ 1  2 35]
[0.19948114 0.06304139 0.03628899]
[26 17 14]
[0.12172861 0.1129551  0.10107455]
[ 0 25 35]
[0.03727398 0.03135575 0.02760048]
[11 41 44]
[0.1269772 0.1150763 0.       ]
[ 3 11 35]
[0.12201161 0.07582256 0.0221057 ]
[25 41 35]
[0.14042891 0.08643272 0.02069205]
[ 2 25 11]
[0.06031172 0.0486961  0.02640242]
[35  0 15]
[0.08893432 0.06355879 0.03550358]
[15 37 35]
[0.19874322 0.04767007 0.03100207]
[34  3 31]
[0.19890256 0.02757125 0.02524587]
[ 1  3 35]
[0.21462147 0.12489849 0.05459066]
[18 31 11]
[0.24943523 0.03178848 0.020419  ]
[40 35 13]
[0.40414593 0.02669308 0.02555969]
[ 7 35 33]
[0.13695698 0.05846167 0.03997829]
[31 21  0]
[0.06867714 0.05917803 0.00608401]
[26 13 31]
[0.18412852 0.11386093 0.08308865]
[31 35 44]
[0.09847656 0.00202264 0.        ]
[41 31 21]
[0.23919893 0.06169803 0.00935534]
[11 41 39]
[0.1227122  0.11942995 0.06363979]
[25  0 23]
[0.14483143 0.07994469 0.02993506]
[38 35 41]
[0.05871938 0.02244531 0.01362826]
[1

[13 11 35]
[0.09639956 0.03048915 0.02645538]
[42  7 38]
[0.26461794 0.07835758 0.05872223]
[27  2 11]
[0.28642354 0.07868119 0.03985411]
[23 35  2]
[0.22998937 0.05468938 0.04224981]
[ 7 35  0]
[0.11318328 0.04794376 0.02108453]
[5 2 0]
[0.11283392 0.08526572 0.06574903]
[ 5  0 23]
[0.0805144  0.05417883 0.00368684]
[32  7  0]
[0.22861188 0.02346696 0.02035851]
[31 24 37]
[0.07719246 0.04146043 0.0302985 ]
[36 31  5]
[0.38351353 0.0559987  0.02743512]
[28 36 37]
[0.0460582  0.03833035 0.03575749]
[39 31 33]
[0.08622027 0.07989    0.00189972]
[27 11 15]
[0.13999427 0.08596445 0.06256272]
[27 12 11]
[0.17855501 0.07771062 0.07246022]
[13 18  0]
[0.14893588 0.13412358 0.0255194 ]
[ 7 19 16]
[0.13566179 0.06920842 0.06481617]
[ 1  2 11]
[0.19408427 0.06670013 0.06611933]
[11  0 21]
[0.04084828 0.03886878 0.02579146]
[ 0 35  9]
[0.08987369 0.03721037 0.01072962]
[11 41  2]
[0.08710321 0.05911303 0.03968647]
[ 4 35  0]
[0.08236247 0.03765267 0.03598531]
[23 35  8]
[0.2364314  0.05847982 0.0

[ 4 12  0]
[0.10539715 0.09241254 0.02770097]
[ 0 35 21]
[0.07658785 0.0516977  0.03188105]
[ 2 35 27]
[0.06070356 0.04724509 0.04291322]
[11 35  2]
[0.06884267 0.03844074 0.03153737]
[39  0 35]
[0.15450816 0.07566582 0.05695601]
[ 6 35  0]
[0.36734109 0.03671587 0.01927013]
[41  0 38]
[0.09884949 0.06096232 0.0488475 ]
[18 23  0]
[0.16034191 0.09784005 0.00861257]
[18  3 37]
[0.29100638 0.1109485  0.02426864]
[40  0  2]
[0.1519117  0.03600992 0.02563664]
[13 30  4]
[0.10857779 0.10217376 0.05981398]
[33  2 35]
[0.23666477 0.02809677 0.01450252]
[21 11 31]
[0.3429914  0.01093027 0.00589641]
[21 31 44]
[0.28293553 0.00604709 0.        ]
[ 0 38  5]
[0.02749282 0.02323727 0.02155242]
[ 0 26 35]
[0.05229414 0.02744539 0.02621224]
[37  0  8]
[0.20415037 0.02246097 0.02027436]
[35 42 41]
[0.05978725 0.03431243 0.02979917]
[ 7 18  0]
[0.11943139 0.02798554 0.02199654]
[ 7 11  2]
[0.08544957 0.03801856 0.03439701]
[24  2  0]
[0.21609345 0.06801609 0.02124535]
[24 41  2]
[0.24304359 0.033357   

[0.05754813 0.03768833 0.00561924]
[ 0 38 21]
[0.13110367 0.08495028 0.02781386]
[ 4  0 28]
[0.16839572 0.05932429 0.01198977]
[0 9 2]
[0.06452745 0.05821199 0.05196148]
[20  0  2]
[0.19665922 0.08013083 0.06838917]
[21  0 31]
[0.06511145 0.0482411  0.02786684]
[ 8  0 38]
[0.15822617 0.09367476 0.03051581]
[ 0 37 28]
[0.11309554 0.0729579  0.01751326]
[ 0 12 25]
[0.2053934  0.03788355 0.035264  ]
[ 8  0 42]
[0.12310369 0.03756326 0.00191036]
[ 0 38  2]
[0.07339781 0.04371678 0.03090205]
[37  0 29]
[0.12440655 0.09014798 0.01881108]
[25  0  2]
[0.24879436 0.08770265 0.00149451]
[25  0 38]
[0.06268032 0.0616253  0.03986511]
[ 0  2 35]
[0.07999482 0.00551803 0.00180787]
[21  0  2]
[0.12592931 0.07816968 0.00662487]
[ 2  0 21]
[0.06677935 0.05885117 0.03432491]
[ 0 21  2]
[0.096086   0.04469665 0.0359892 ]
[ 0 28 37]
[0.07249487 0.02732617 0.02221991]
[ 0 38 35]
[0.03932757 0.02062047 0.01412282]
[15  0  6]
[0.21122774 0.09946834 0.01446541]
[20  0 25]
[0.18685671 0.10497295 0.05385397]
[5

[0.29739489 0.03835848 0.        ]
[ 0  8 23]
[0.19977746 0.06181652 0.04299803]
[33  7  2]
[0.16599256 0.08987784 0.08061834]
[ 0  2 35]
[0.07755338 0.03237192 0.03070271]
[ 4 30  0]
[0.22329209 0.07160475 0.03569105]
[ 2  0 25]
[0.08568463 0.07996944 0.02278977]
[23  2 31]
[0.08838368 0.07106872 0.03427384]
[23 25  4]
[0.09183222 0.08313706 0.07825812]
[2 0 7]
[0.09598204 0.04229538 0.04009475]
[ 0 37 35]
[0.1180827  0.04536877 0.02162244]
[ 0 38  8]
[0.10511027 0.03286337 0.02749897]
[15  2  0]
[0.13474525 0.03083675 0.02965513]
[15  0 38]
[0.18011623 0.10928434 0.02459208]
[ 9 29 21]
[0.30091194 0.04504917 0.03954677]
[22  2 35]
[0.15465253 0.13932224 0.01757775]
[25 39 14]
[0.1813171  0.05723272 0.02393612]
[ 0  7 37]
[0.10738068 0.02915175 0.02728344]
[ 2 25  0]
[0.04214652 0.03736622 0.02862955]
[ 0 23 24]
[0.07601885 0.02459604 0.00251286]
[ 0 40 23]
[0.13902437 0.13053177 0.04731174]
[40  2 11]
[0.15945976 0.09856505 0.0499126 ]
[27  2  8]
[0.29685626 0.08171087 0.01479786]
[2

[7 2 0]
[0.11109923 0.04993882 0.015908  ]
[ 0 16 37]
[0.14445205 0.05147356 0.04813488]
[ 2  0 38]
[0.1102248  0.02193973 0.02053489]
[23 35  2]
[0.1444411  0.07062313 0.06367511]
[ 0 16  3]
[0.14379967 0.0348002  0.00826599]
[16 31  0]
[0.08615912 0.01162799 0.00752661]
[ 0 16  3]
[0.06951609 0.04725525 0.03696993]
[ 0  2 11]
[0.07856779 0.05604917 0.02520219]
[ 0  8 12]
[0.20531474 0.05217058 0.05016387]
[ 2 20 29]
[0.15537229 0.0244507  0.01466335]
[0 2 4]
[0.02933972 0.01924354 0.00361875]
[15  2  3]
[0.17388285 0.0355793  0.02945   ]
[ 3  2 15]
[0.25103838 0.07582131 0.07128695]
[ 0 12 23]
[0.15916369 0.05280383 0.01686676]
[23  0  2]
[0.23146238 0.09897512 0.03553014]
[31 35 11]
[0.01812905 0.01438919 0.00400977]
[ 1  2 11]
[0.25933188 0.07472481 0.02349889]
[ 4 30  3]
[0.10939784 0.08004073 0.06892008]
[7 2 0]
[0.17737956 0.03513145 0.02064419]
[ 2  0 11]
[0.08513482 0.02452685 0.01383385]
[ 0 38 37]
[0.07523271 0.07131821 0.03392284]
[ 7  0 38]
[0.17062683 0.05999238 0.0379381

In [268]:
nums_rate = dict(sorted(dict(Counter(list(df2.loc[:, 'topic_number']))).items(), key=lambda kv: kv[1], reverse=True))
len(nums_rate)

45

In [269]:
nums_rate

{31: 264,
 16: 157,
 38: 150,
 7: 136,
 10: 124,
 5: 119,
 23: 115,
 9: 104,
 8: 101,
 25: 94,
 37: 93,
 30: 91,
 15: 81,
 13: 79,
 34: 78,
 21: 75,
 19: 72,
 4: 71,
 18: 69,
 3: 69,
 28: 63,
 27: 60,
 20: 57,
 22: 57,
 43: 55,
 12: 53,
 24: 51,
 2: 48,
 39: 47,
 33: 45,
 29: 45,
 26: 43,
 40: 41,
 14: 40,
 11: 37,
 36: 36,
 32: 35,
 17: 34,
 1: 34,
 6: 33,
 44: 32,
 0: 28,
 42: 27,
 35: 17,
 41: 8}

In [270]:
df3 = get_topics(sorted_dfs[2], 38, 15, ['nmf'], [0, 3]) # 2013 - 2016

NMF
Topic 0:
развитие международный сотрудничество регион интерес экономический мир работа новый безопасность основа усилие мировой глобальный укрепление
Topic 1:
сирийский сирия оппозиция боевик террорист алеппо дамаск сара вооруженный группировка район джабхат террористический ан-нусра правительство
Topic 2:
украина украинский киев власть киевский юго-восток минский заявление ситуация действие донецкий донбасс донецк миссия сила
Topic 3:
визит сотрудничество двусторонний федерация вопрос актуальный ход республика переговоры взаимодействие повестка обсуждать состояться рабочий посещать
Topic 4:
погибший выздоровление соболезнование пострадавший осуждать террористический решительно погибать выражать близкие скорый родные теракт ранение результат
Topic 5:
нато альянс военный безопасность блок европа угроза натовский североатлантический россия-нато учение черногория сила европейский заявление
Topic 6:
оон резолюция сб совет безопасность генеральный право международный принимать сессия чл

[12  0 17]
[0.06752799 0.0192047  0.01175976]
[20 14  1]
[0.2303045  0.04258782 0.01804487]
[5 0 7]
[0.23520609 0.04660829 0.010917  ]
[ 3 35 36]
[0.08484127 0.04003312 0.01821568]
[ 2 17  9]
[0.08020717 0.01363915 0.00775904]
[19  6 12]
[0.07461123 0.04828412 0.02529059]
[ 0 15 23]
[0.03630902 0.01752928 0.00821395]
[10  6  0]
[0.28901374 0.01183558 0.01109805]
[ 4 31  0]
[0.14865274 0.02535974 0.01996674]
[11 14  5]
[0.13052274 0.04616199 0.01864916]
[ 4 14 21]
[0.17502802 0.05751331 0.01442137]
[19  0  2]
[0.15123626 0.02774818 0.01998065]
[18 14 30]
[0.15106017 0.14136172 0.0053189 ]
[24 14 34]
[0.19039683 0.05254093 0.02957521]
[ 0 27  3]
[0.06605314 0.02405016 0.01917856]
[31  4  6]
[0.24932662 0.04616386 0.03471425]
[ 4 32  1]
[0.15905762 0.13865232 0.02291123]
[37  0  1]
[0.08530132 0.02619179 0.01753567]
[19 12  0]
[0.08180143 0.03378687 0.01451487]
[ 4 21  1]
[0.18542625 0.05673389 0.03395902]
[24 14  0]
[0.14889666 0.04793001 0.01808903]
[37  3  0]
[0.20690383 0.07532342 0.0

[0.09935146 0.04604287 0.01241613]
[23  0  7]
[0.06201928 0.02206352 0.01904832]
[ 6  0 14]
[0.1034502  0.02078077 0.01938834]
[31 14  0]
[0.36963984 0.03079122 0.01419086]
[34 14 36]
[0.17204968 0.08237762 0.01597814]
[7 2 0]
[0.04104825 0.03227944 0.01291164]
[ 3 37  0]
[0.08071916 0.06605425 0.04787014]
[12  0  3]
[0.17212558 0.02123363 0.0144704 ]
[5 0 6]
[0.08769413 0.04590811 0.0392928 ]
[ 7  0 23]
[0.08816027 0.03475665 0.02911002]
[37  3  0]
[0.11282851 0.04176039 0.03376963]
[ 7 14  0]
[0.1039608  0.07601846 0.00906127]
[36  2  6]
[0.17201057 0.07145291 0.0401745 ]
[31 14  0]
[0.33936999 0.03573924 0.02095603]
[5 7 0]
[0.13874478 0.09089027 0.01637758]
[ 4 14 12]
[0.11138693 0.02335438 0.00802614]
[21  3  0]
[0.18300859 0.08082166 0.05071275]
[30  2 14]
[0.0659418  0.05404341 0.04179752]
[ 9 17 11]
[0.0903416  0.01146673 0.00692849]
[21 15  4]
[0.08134385 0.07835084 0.02737615]
[16  3  0]
[0.1105125  0.10634141 0.04712312]
[0 5 3]
[0.04774916 0.0218286  0.01207782]
[22 14 36]


[0.05380286 0.01944308 0.01771284]
[ 1  6 30]
[0.08730434 0.08639013 0.03529657]
[22  6  7]
[0.06429371 0.05232984 0.0432179 ]
[ 2 25  0]
[0.08216429 0.02263742 0.01970943]
[ 5  2 13]
[0.07816599 0.07018489 0.03671482]
[ 1 24  4]
[0.15646808 0.04083978 0.03600054]
[ 2 19  3]
[0.0309301  0.03065182 0.0270134 ]
[ 6  7 30]
[0.07954012 0.05706563 0.04729892]
[ 2  6 22]
[0.0974597  0.08704455 0.02899283]
[ 0 23  2]
[0.03142537 0.02351643 0.00488348]
[37  0 25]
[0.02401307 0.01839768 0.00932211]
[ 2  9 11]
[0.08981179 0.02291122 0.01862651]
[ 2 14 24]
[0.06696938 0.04780877 0.01151161]
[ 2 12 37]
[0.10557349 0.00440273 0.        ]
[ 1 24  6]
[0.16413752 0.11135376 0.02315292]
[22  2  0]
[0.0920587  0.04073002 0.02200556]
[ 2 14 37]
[0.06082083 0.03385427 0.        ]
[23  0  7]
[0.19903477 0.02976397 0.01109606]
[13  2 22]
[0.08795044 0.06389133 0.04052734]
[ 1 32 36]
[0.139322   0.04314099 0.04104322]
[ 7 22  0]
[0.10774603 0.07274171 0.00341957]
[ 2 22  0]
[0.07163314 0.04693053 0.01035618]

[29 23  0]
[0.29933054 0.06041635 0.02767673]
[14  0 19]
[0.11148569 0.01579384 0.01410493]
[25 13  0]
[0.23058092 0.01716476 0.01026591]
[ 0  3 35]
[0.041923   0.02485463 0.01676106]
[ 3 37  9]
[0.17485958 0.         0.        ]
[ 6  1 28]
[0.14663451 0.06294096 0.05275211]
[ 1 36 21]
[0.12291226 0.04952698 0.02572649]
[7 1 6]
[0.07109972 0.02653682 0.01581931]
[21  1  7]
[0.1146799  0.04266794 0.02501777]
[33  0  6]
[0.19078956 0.04266726 0.03571635]
[ 7 30 33]
[0.06894641 0.06275066 0.06060601]
[34 30 14]
[0.22682416 0.07710442 0.06528921]
[ 0 14  1]
[0.04903504 0.01357658 0.00709564]
[25  0  3]
[0.03895577 0.02442522 0.01620415]
[ 1 36  3]
[0.10019006 0.08593936 0.04703289]
[23  3 21]
[0.12067548 0.07310557 0.0413052 ]
[21  1  3]
[0.08013894 0.06138165 0.03880703]
[ 3 19  9]
[0.14223261 0.00844749 0.        ]
[26  3 37]
[0.24860817 0.10821212 0.        ]
[0 3 6]
[0.0548513  0.03155764 0.02287985]
[30  1 25]
[0.08694838 0.04357169 0.03723182]
[ 1 36  7]
[0.13169695 0.09962659 0.0457

...

[ 7 30 25]
[0.05996256 0.0363202  0.02488849]
[ 7  1 30]
[0.06786088 0.05584326 0.03423473]
[10  4 14]
[0.30307443 0.02032347 0.01734293]
[8 6 0]
[0.25710406 0.03875185 0.03456666]
[20  6 16]
[0.33039769 0.07074089 0.00901471]
[23  6 35]
[0.15813019 0.12085773 0.02234496]
[18  6 35]
[0.21085622 0.02075271 0.01773416]
[36  2  0]
[0.18593409 0.05435564 0.03545973]
[35  3 13]
[0.05830184 0.04042729 0.03017678]
[33 36  4]
[0.24667058 0.02439877 0.02167405]
[0 6 3]
[0.09034722 0.01414782 0.00774684]
[14 34  0]
[0.02250555 0.01636402 0.01626161]
[13  3 28]
[0.23440053 0.05157485 0.03892606]
[ 3 16 36]
[0.13358429 0.05922465 0.01158608]
[ 3 28 27]
[0.06149898 0.04260384 0.02273075]
[27  3  0]
[0.38235875 0.03270294 0.00735897]
[ 3  0 27]
[0.05701268 0.02500315 0.02086067]
[25 13  0]
[0.23972399 0.04391046 0.01088521]
[35 12  2]
[0.06029855 0.04514874 0.04485171]
[13  2 36]
[0.08072569 0.07367697 0.02182831]
[ 1  0 21]
[0.09994217 0.04793963 0.04327114]
[6 0 1]
[0.13084411 0.04983412 0.0204976

[ 3 35 14]
[0.06245103 0.01162134 0.006041  ]
[4 0 3]
[0.23523173 0.00385411 0.00113122]
[20  3  1]
[0.29588766 0.1084368  0.0043082 ]
[24  3 28]
[0.13065918 0.10161999 0.03765916]
[37  3  0]
[0.17008149 0.06449752 0.04393865]
[ 0 27  3]
[0.05030132 0.02402166 0.02312731]
[ 3 14 22]
[0.10773511 0.0083205  0.00255545]
[ 3 12  0]
[0.08236898 0.06809975 0.04118037]
[ 2 25 13]
[0.1261566  0.09446255 0.0823657 ]
[13  2 30]
[0.12859191 0.06749531 0.02830108]
[ 0 37 12]
[0.05267842 0.04630046 0.01076297]
[18  4 14]
[0.17807865 0.03178305 0.02862514]
[8 4 1]
[0.28558843 0.04282857 0.03075964]
[36 14 13]
[0.11423695 0.02477633 0.01686668]
[14  0 12]
[0.05454226 0.01419489 0.01372281]
[16 17 35]
[0.02903086 0.01822876 0.01747241]
[ 0 33  3]
[0.08396909 0.03747228 0.01619507]
[0 3 9]
[0.06401435 0.02921241 0.00451909]
[16  0  3]
[0.27724175 0.01755321 0.00436508]
[30  2  4]
[0.14629694 0.05255833 0.04626042]
[ 2 22 25]
[0.14561227 0.04830354 0.03461025]
[ 2  0 35]
[0.07864965 0.03566303 0.0096833

[ 9  0 35]
[0.02517011 0.02266646 0.01169573]
[35  6  1]
[0.10018838 0.09046452 0.07850267]
[ 0  9 35]
[0.08694177 0.04192107 0.01448943]
[35  6  3]
[0.06230565 0.05965919 0.02471367]
[ 0 17  1]
[0.09318867 0.02072221 0.        ]
[35  0  6]
[0.06137408 0.05174353 0.04940709]
[35  9  3]
[0.10005217 0.06897451 0.03583395]
[ 1 36 35]
[0.14735869 0.07274946 0.06288344]
[35  1  6]
[0.0845995  0.08284554 0.04227506]
[0 3 9]
[0.06153986 0.00936906 0.        ]
[35  1  3]
[0.14168884 0.01028672 0.00652955]
[0 9 3]
[0.07592331 0.01530134 0.00745908]
[16  0  3]
[0.14306871 0.04853534 0.047789  ]
[ 0 17 27]
[0.09142845 0.06955389 0.0645232 ]
[19  0 35]
[0.14067956 0.11636901 0.03334792]
[37  0  3]
[0.14628682 0.06420823 0.04045026]
[ 0 21 35]
[0.07879757 0.03483682 0.03023454]
[ 3  0 37]
[0.10738498 0.07507198 0.06747426]
[37  0  3]
[0.10078921 0.08119895 0.05326711]
[35  3  6]
[0.08682296 0.03629293 0.01006793]
[ 0 27 22]
[0.10607173 0.08965432 0.02799351]
[0 3 9]
[0.0472033  0.04459359 0.0401610

[0.0561544  0.03750019 0.01042692]
[ 9 13 35]
[0.06709333 0.05459603 0.04064878]
[35  9  0]
[0.13528703 0.0452582  0.03961113]
[ 9 35  0]
[0.09933949 0.03671095 0.03195215]
[35  0  9]
[0.04457288 0.03016731 0.01027073]
[ 9  0 35]
[0.11917595 0.06460766 0.01990868]
[ 0  9 16]
[0.04435583 0.         0.        ]
[9 0 6]
[0.26701293 0.04071098 0.01418301]
[ 9 19  0]
[0.07519819 0.05980139 0.05694828]
[ 9 35  0]
[0.21285413 0.02950668 0.02611375]
[ 0  9 17]
[0.09112336 0.01232953 0.00893949]
[ 0 27 22]
[0.11092779 0.06006713 0.05918026]
[0 9 2]
[0.0863546  0.07485473 0.02376114]
[37  3 35]
[0.07896806 0.06111897 0.05561948]
[6 0 9]
[0.12568283 0.12460131 0.01815694]
[16  0  3]
[0.17076239 0.07029986 0.02758323]
[37  9  0]
[0.25181653 0.09031949 0.05211606]
[0 5 2]
[0.15245268 0.04497739 0.0400918 ]
[35  2 13]
[0.11273139 0.0727832  0.05028656]
[23 35  0]
[0.16947361 0.04479214 0.0233162 ]
[ 0 25 35]
[0.04732136 0.04511778 0.01147567]
[ 0  9 17]
[0.17087868 0.02991957 0.02425484]
[ 1  0 35]


[ 1 35  0]
[0.08479715 0.08257418 0.02898757]
[ 0 17  9]
[0.11491268 0.08178309 0.02028745]
[30 23  3]
[0.07382864 0.04398175 0.04132209]
[ 1 35  0]
[0.08271748 0.05859265 0.02562565]
[ 0 17  1]
[0.15927477 0.0093561  0.        ]
[29  0 35]
[0.09473869 0.07607385 0.03964124]


In [271]:
df4 = get_topics(sorted_dfs[3], 47, 15, ['nmf'], [4, 0, 13, 23]) # С 2016

NMF
Topic 0:
рад уважаемый друг пожаловать добро сегодня дорогой президент возможность ценить хороший встреча коллега встречаться контакт
Topic 1:
сша американский вашингтон санкция американец госдепартамент администрация против заявление санкционный собственный давление обвинение военный попытка
Topic 2:
сирийский сирия турция астанинский процесс урегулирование оон резолюция иран конгресс сб сочи оппозиция астана встреча
Topic 3:
украина украинский киев киевский донбасс власть минский закон режим порошенко миссия действие нынешний вышинский очередной
Topic 4:
визит двусторонний сотрудничество переговоры республика федерация обсуждать актуальный региональный вопрос рабочий повестка взаимодействие обмен ход
Topic 5:
война советский память памятник история второй победа великий исторический мировой нацизм отечественный чешский фашизм освобождение
Topic 6:
сирия сирийский террорист боевик провинция гуманитарный район зона житель лагерь пункт военный алеппо помощь дамаск
Topic 7:
одкб безо

[30 38 26]
[0.06576848 0.04740601 0.028735  ]
[10 12 30]
[0.17364668 0.01817538 0.01129062]
[13 21 35]
[0.06940451 0.03250005 0.02469883]
[ 9 13  4]
[0.11247931 0.03327152 0.02412112]
[20  8 13]
[0.18022217 0.12160976 0.00869761]
[24  2 45]
[0.11105219 0.0783781  0.0761667 ]
[ 5 38 10]
[0.04917511 0.02826604 0.02709741]
[40  2 17]
[0.07237821 0.04842828 0.0473981 ]
[13 29 40]
[0.03848555 0.02629343 0.02352407]
[40 13 32]
[0.04521679 0.02984548 0.0281751 ]
[23  4 13]
[0.05559501 0.05227519 0.02146516]
[38 32  1]
[0.04884871 0.04549038 0.00932665]
[ 3 19 32]
[0.11172406 0.05594942 0.04460972]
[40 13 18]
[0.13019115 0.06035027 0.02549187]
[32 23 13]
[0.13374213 0.0601924  0.0353892 ]
[ 8  6 13]
[0.15534342 0.01557103 0.01540809]
[32 40  4]
[0.16706068 0.04391832 0.01473096]
[31  6 13]
[0.30033655 0.02207961 0.02155565]
[1 6 2]
[0.09739904 0.04128845 0.02494684]
[ 1  5 13]
[0.08422415 0.02220102 0.02203656]
[27 13  1]
[0.26053501 0.03633398 0.01847568]
[ 4 13  3]
[0.10567816 0.03982593 0.0

[ 6 13  2]
[0.09229841 0.02909366 0.01186329]
[32 13 19]
[0.08333791 0.02273689 0.00705196]
[35 13 40]
[0.08374493 0.04246074 0.0404418 ]
[15 13 29]
[0.2939124  0.05526236 0.01998006]
[ 6 13  1]
[0.07853023 0.02765955 0.02408547]
[32 23 13]
[0.12163056 0.06316934 0.03579601]
[25  4 13]
[0.12281332 0.08204534 0.02958986]
[30 33 20]
[0.1002371  0.04277548 0.01452038]
[27 40 13]
[0.16807744 0.05127876 0.03111315]
[10  5 13]
[0.21037134 0.08916103 0.01011894]
[16  4 23]
[0.27528139 0.07680992 0.03351581]
[ 5  1 23]
[0.09962292 0.03689371 0.02885191]
[ 7 18 23]
[0.16610404 0.15008029 0.02713871]
[ 2 40  4]
[0.10739438 0.05759705 0.02544633]
[23  5 32]
[0.08652519 0.04827515 0.00826613]
[ 4  9 41]
[0.12224964 0.05580504 0.0539893 ]
[36 23 13]
[0.02599487 0.02424799 0.01781264]
[ 6 13  2]
[0.20852609 0.02728771 0.02026903]
[12  6  2]
[0.08481758 0.07925388 0.02313271]
[ 8 13  6]
[0.14658154 0.03034256 0.01740929]
[11 13  1]
[0.23561793 0.0520208  0.0479249 ]
[29 13 30]
[0.09760794 0.05725446 

[0.0774471  0.04333452 0.02931773]
[ 4  9 14]
[0.12836549 0.04901745 0.02340016]
[35  1 13]
[0.1933749  0.07468823 0.03649936]
[ 5 37 13]
[0.17778531 0.13758786 0.04453279]
[16  5  1]
[0.13368187 0.06175447 0.05147128]
[41 19 13]
[0.30763749 0.04047268 0.0256662 ]
[28 30 20]
[0.04778709 0.02285398 0.02255983]
[28 30 13]
[0.05665833 0.03103488 0.01676901]
[34 13 15]
[0.21258289 0.03558635 0.02496044]
[16 13 23]
[0.11905776 0.02817529 0.01727692]
[16 13 19]
[0.10025824 0.01948304 0.01043546]
[13  7 11]
[0.03302245 0.02889493 0.0163634 ]
[31 13  6]
[0.2945302  0.03291282 0.0082429 ]
[13  1  4]
[0.04075204 0.01189441 0.01138397]
[23 44 19]
[0.0803626  0.07890725 0.00151597]
[ 6  2 45]
[0.12344597 0.11362427 0.01684307]
[13 11  9]
[0.04538992 0.01950031 0.01497696]
[13  1 17]
[0.04739995 0.04419282 0.03719022]
[30 17 22]
[0.11635995 0.07688585 0.02484253]
[11  1 13]
[0.17890158 0.07754747 0.03430478]
[ 5 19 13]
[0.06874283 0.0220318  0.01988216]
[22 19 13]
[0.14856086 0.04342198 0.01770705]

[0.12508143 0.04568865 0.00786958]
[ 4 14 13]
[0.05984412 0.05659822 0.03841318]
[42  4  7]
[0.20961341 0.10893645 0.06964237]
[ 9  4 14]
[0.15205342 0.09380322 0.02585183]
[45  4 13]
[0.21898791 0.0922845  0.01039968]
[6 2 1]
[0.11718457 0.10486707 0.02220994]
[19  3 30]
[0.08032839 0.07864493 0.03480244]
[32  3  0]
[0.18440739 0.06576814 0.02067646]
[ 3 32 13]
[0.08527311 0.06985776 0.029845  ]
[19 30  3]
[0.05467241 0.02832085 0.02045071]
[22 13 19]
[0.29767154 0.01737785 0.01608634]
[ 1 13 32]
[0.05389177 0.02007833 0.01077491]
[19 26 15]
[0.16381935 0.08851028 0.07501181]
[22 19 13]
[0.08503913 0.07159096 0.0136369 ]
[23 24 14]
[0.03553359 0.02546341 0.02211866]
[ 5 22 13]
[0.11671016 0.04000562 0.02380684]
[8 1 6]
[0.22317863 0.04623382 0.02208871]
[17 35  2]
[0.06853587 0.06207735 0.04619426]
[16  4 35]
[0.15461099 0.05654325 0.02552183]
[30  1 44]
[0.13844286 0.02176442 0.00879051]
[30  1 35]
[0.12325237 0.09501747 0.01795957]
[ 1 30 13]
[0.08190713 0.02844719 0.00918786]
[ 1 3

...

[ 5 23 32]
[0.08154881 0.04240737 0.01184299]
[23  5 30]
[0.04677783 0.02251052 0.00797515]
[23 13  4]
[0.03222981 0.02124235 0.00554028]
[ 1 19 26]
[0.10060077 0.09162105 0.0178365 ]
[19  1 13]
[0.06799128 0.03308962 0.03081543]
[16  4 23]
[0.12590243 0.05380891 0.04296229]
[40 23  2]
[0.06507808 0.05902658 0.04100489]
[13 23  4]
[0.03894063 0.03302438 0.01648726]
[36 33 32]
[0.29438315 0.17594922 0.06289661]
[ 7  4 40]
[0.36484909 0.01395123 0.00994712]
[ 4 27 15]
[0.09082248 0.05885401 0.03963338]
[23 40  8]
[0.04697967 0.03322025 0.02210162]
[ 6  2 23]
[0.12773703 0.10083265 0.01425498]
[19  2 13]
[0.05894074 0.04078912 0.0261708 ]
[12 13 40]
[0.14104277 0.03671425 0.02280645]
[13 29  4]
[0.03602661 0.02152669 0.00865517]
[13  9  4]
[0.0276405  0.0164104  0.01550844]
[28 30 13]
[0.03291983 0.03094815 0.02893221]
[28  1 19]
[0.07857431 0.06203732 0.01887753]
[5 3 1]
[0.08715319 0.05965129 0.03964007]
[19  5 13]
[0.03831044 0.03712336 0.03149447]
[38 32 23]
[0.3387907  0.01562292 0.0

[ 1 29 19]
[0.10284291 0.02271346 0.01917675]
[30 34 32]
[0.14427353 0.03210813 0.00807031]
[34 36 23]
[0.086546   0.02682956 0.01917544]
[ 5 30 20]
[0.09906498 0.01928527 0.01503885]
[13 19  1]
[0.02817708 0.00680503 0.00559163]
[ 5 46 23]
[0.09744528 0.02324992 0.00702047]
[46  5  6]
[0.2260384  0.12701408 0.01237309]
[ 5 46  3]
[0.04565307 0.04099498 0.03720578]
[46  3 13]
[0.19759465 0.0911894  0.02112623]
[ 6 12  2]
[0.19763959 0.06066948 0.02476598]
[ 8 20  6]
[0.19833497 0.02445134 0.01813545]
[10  5 13]
[0.18605465 0.06190069 0.02592637]
[30 13  2]
[0.02251496 0.01951124 0.01924363]
[ 3 46 30]
[0.08519264 0.07814944 0.04382653]
[19 33 30]
[0.03430626 0.02874184 0.02465346]
[46 23  5]
[0.12962041 0.08877443 0.02026494]
[ 5 33 19]
[0.07467409 0.04528457 0.01634148]
[46  5 13]
[0.22175519 0.01405633 0.01159773]
[34 15  2]
[0.15319506 0.10316707 0.01638348]
[43  4 46]
[0.09511147 0.09435357 0.        ]
[43 23  4]
[0.40241191 0.02719612 0.02115213]
[ 5 33 23]
[0.17408598 0.10454851 

[40  0 23]
[0.09276356 0.04949193 0.02064659]
[13 35  0]
[0.11742134 0.05100345 0.02504124]
[13 35 17]
[0.10359334 0.08286172 0.04160725]
[ 9 13 40]
[0.10039811 0.09121384 0.03262719]
[ 0  9 40]
[0.10638294 0.067884   0.0269321 ]
[ 9 13 23]
[0.20015125 0.07300328 0.02320727]
[13  4 11]
[0.08035966 0.05145053 0.02448612]
[ 0 38 40]
[0.12599943 0.05480183 0.04109625]
[15 13 32]
[0.15199345 0.06443525 0.04450044]
[35 13 15]
[0.12404053 0.08103006 0.07979403]
[ 4 15 23]
[0.08446031 0.07638809 0.02526439]
[15 43  4]
[0.11081427 0.08331515 0.0654775 ]
[13  5 44]
[0.04558723 0.04454711 0.01756566]
[ 2 13  0]
[0.16485778 0.04001399 0.03773999]
[24  4 13]
[0.05665843 0.0558864  0.02597765]
[ 0 13 40]
[0.04914682 0.04612461 0.02763202]
[ 4 23 40]
[0.05103902 0.02552471 0.0223346 ]
[ 0 23 13]
[0.09152879 0.04836892 0.01503113]
[ 4 17 14]
[0.11994447 0.02488447 0.0210271 ]
[13 17  0]
[0.09765432 0.0309474  0.02809458]
[ 0 20 13]
[0.09687374 0.05202414 0.03405507]
[40 31 30]
[0.07349486 0.04611771 

[ 0 15  4]
[0.09295797 0.03043218 0.01117942]
[13  3 15]
[0.06591639 0.0406017  0.03764576]
[13  0 23]
[0.03861763 0.03520786 0.02250766]
[12 26 22]
[0.22431609 0.04758488 0.04029738]
[13  4  0]
[0.09892956 0.02648595 0.01546019]
[ 0 13 40]
[0.06830231 0.04411534 0.01407983]
[27  4 13]
[0.23725548 0.04305612 0.0267454 ]
[27  0 23]
[0.18846126 0.07262093 0.04263054]
[13 30 32]
[0.05818356 0.03437161 0.0167168 ]
[ 7 13 37]
[0.10719304 0.06533981 0.04183484]
[37  0  7]
[0.16079826 0.07457852 0.06388123]
[37 13 15]
[0.07744894 0.0433861  0.03130639]
[18 13  2]
[0.12944104 0.05844833 0.05687782]
[18  0 13]
[0.13707267 0.061879   0.03935128]
[13 35 12]
[0.12688191 0.05939713 0.04591072]
[ 0  4 13]
[0.09565485 0.03953074 0.01290286]
[ 6  2 40]
[0.12821529 0.10588244 0.05139502]
[ 2  0 40]
[0.12485791 0.0828268  0.02979109]
[ 7  0 37]
[0.06076599 0.05151909 0.04715314]
[ 8 13 42]
[0.16819193 0.10040526 0.01650251]
[36 13 44]
[0.38544895 0.07863267 0.04924192]
[ 0 13 25]
[0.05242714 0.03078672 

[0.21811696 0.03692961 0.02096185]
[ 0 20  9]
[0.09950259 0.06961922 0.06055017]
[44  5 20]
[0.07117617 0.03565698 0.0226284 ]
[13 35 12]
[0.06123466 0.04296188 0.03147452]
[34 23  0]
[0.23952204 0.02892116 0.02730774]
[34 13  4]
[0.32055871 0.04196969 0.03294211]
[ 0  5 13]
[0.14819116 0.00205945 0.00191568]
[ 0 13 40]
[0.03618329 0.03340169 0.03257448]
[ 0 13 15]
[0.07670374 0.03744705 0.02134298]
[ 4 37 23]
[0.09411068 0.0336155  0.02712108]
[13 43  0]
[0.0821963  0.07889031 0.01778077]
[ 0 44 13]
[0.0548625  0.03974559 0.03513451]
[ 9 13  4]
[0.17353765 0.06792148 0.03250355]
[ 0 13  4]
[0.0940371  0.0163515  0.01390776]
[13  3 26]
[0.05565219 0.05182076 0.04252254]
[13  5  1]
[0.1321322  0.02671825 0.01812057]
[29  0  1]
[0.07257897 0.04857358 0.00342449]
[13  5 46]
[0.0357033  0.02007406 0.01579947]
[16  0  4]
[0.15802024 0.08147746 0.04637899]
[ 0 13 11]
[0.05478811 0.00195552 0.        ]
[ 1 13  0]
[0.07699257 0.03777106 0.03372907]
[ 0 23 13]
[0.07992791 0.03493976 0.00843406]

In [272]:
topics_labels = get_json_from_file('topics_labels.json')
topics_labels_upd = {}
for indx, label_to_topics in topics_labels.items():
    topic_to_label = {}
    for label, topics in label_to_topics.items():
        for topic in topics:
            topic_to_label[topic] = label
    topics_labels_upd[indx] = topic_to_label
    
dfs = [df1, df2, df3, df4]

n = 0
for df in dfs:
    print('Dataframe', n)
    nums_rate = dict(sorted(dict(Counter(list(df.loc[:, 'topic_number']))).items(), key=lambda kv: kv[1], reverse=True))
    labels_rate = {}
    topic_to_label = topics_labels_upd[n]
    for topic, freq in nums_rate.items():
        label = topic_to_label[topic]
        if label not in labels_rate:
            labels_rate[label] = freq
        else:
            labels_rate[label] += freq
    print(sorted(labels_rate.items(), key=lambda kv: kv[1], reverse=True))
    n += 1

Dataframe 0
[('Постсов', 290), ('Ближний Восток', 219), ('Европа', 180), ('Ядерная безопасность', 157), ('Терроризм', 138), ('Экономика', 117), ('ВОВ', 73), ('Азия', 45), ('Исламский мир', 44), ('Арктика', 44), ('Африка', 43), ('Права человека', 41), ('Русская культура', 40), ('НАТО', 39), ('Общие темы', 33), ('Соотечественники за рубежом', 32), ('НПО', 24), ('ЮНЕСКО', 24)]
Dataframe 1
[('Постсов', 502), ('Ближний Восток', 468), ('Инциденты с участием россиян', 406), ('Азия', 260), ('Ядерная безопасность', 230), ('Европа', 196), ('ВОВ', 157), ('ООН', 150), ('Общие темы', 138), ('США', 119), ('Право', 93), ('Русская культура', 91), ('Африка', 86), ('Гуманитарная помощь', 75), ('Соотечественники за рубежом', 71), ('Выборы', 69), ('Арктика', 57)]
Dataframe 2
[('Постсов', 932), ('Ближний Восток', 493), ('Ядерная безопасность', 289), ('Соотечественники за рубежом', 234), ('Европа', 195), ('ВОВ', 179), ('Африка', 168), ('Терроризм', 163), ('ООН', 112), ('Латинская Америка', 102), ('Гуманитар

In [211]:
for indx, row in df4.iterrows():
    if row['topic_number'] in [5]:
        print(row['text'], '\n')

Сегодня, 27 января 2020 года, в день 76-й годовщины полного освобождения Ленинграда от немецко-фашистской блокады по линии Комитета по внешним связям Правительства Санкт-Петербурга и МИД ФРГ подписано рамочное «Совместное заявление о намерениях» о реализации добровольного гуманитарного жеста Правительства ФРГ в адрес ныне здравствующих блокадников Ленинграда, закрепляющее основные административно-правовые принципы осуществления на практике этой инициативы германской стороны. Заключен также рамочный «Договор о техническом сотрудничестве», регулирующий ряд финансовых и организационных аспектов этого процесса.
Начинается практическая реализация договорённости, зафиксированной в Совместном заявлении Министра иностранных дел России С.В.Лаврова и Федерального министра иностранных дел Германии Х.Мааса от 27 января 2019 года. Предшествовали этому переговоры с германской стороной, которые велись по линии МИД России с участием Администрации Санкт-Петербурга.
В рамках Жеста Правительством ФРГ выд



25 ноября на Украине отмечают день памяти жертв «голодомора», погибших в СССР в 30-е гг. XX в. В Киеве настойчиво пытаются представить эти трагические события как «геноцид украинского народа». Такая трактовка не имеет ничего общего с восстановлением справедливости, носит политизированный и антиисторический характер. На наш взгляд, когда так ставится вопрос и подается тема, цель одна – посеять раздор между народами России и Украины.
Хотела бы напомнить, что в начале 30-х гг. XX в. голод, возникший в результате засухи и насильственной коллективизации и раскулачивания, охватил, помимо Украины, Белоруссию, Казахстан, Поволжье, Северный Кавказ, Западную Сибирь и Южный Урал. По разным оценкам, погибло от 2 до 8 млн. человек. Эти события являются общей трагедией русских, украинцев, казахов и других народов страны и, конечно, крупнейшей гуманитарной катастрофой на территории СССР. Поэтому все разговоры о некой целенаправленной политике уничтожения украинской нации противоречат историческим ф

### Sorting data by unofficial dates

In [256]:
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs2 = sort_by_unofficial_dates(df)
get_stat(sorted_dfs2)

Corpus 0: 199 texts 78201 words
Corpus 1: 1228 texts 691187 words
Corpus 2: 3849 texts 1372091 words
Corpus 3: 7096 texts 2318423 words


In [257]:
for df in sorted_dfs2:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [258]:
get_topics(sorted_dfs2[0].loc[:, 'text_clean'], 20, 15, ['nmf']) # До 2004

NMF
Topic 0:
сотрудничество развитие международный работа проблема вопрос организация новый проект мировой важный взаимодействие сфера деятельность регион
Topic 1:
палестинский дорожный карта израиль пна четверка израильский посредник ближневосточный палестинец палестино-израильский урегулирование правительство возобновление мир
Topic 2:
ирак иракский санкция ситуация гуманитарный поездка ущерб посольство заместитель программа война гражданин работать вопрос экономический
Topic 3:
комиссия социальный член гражданин организация заседание программа история федерация научный возможность господин служба утверждение семья
Topic 4:
договор стратегический ратификация сша государственный федеральный федерация российский-американский собрание стабильность президент комитет потенциал близкий фактор
Topic 5:
террористический осуждать соболезнование акт решительно акция погибший жертва выражать взрыв сочувствие сообщение результат иметься терроризм
Topic 6:
транзит калининградский документ соглаше

In [259]:
get_topics(sorted_dfs2[1].loc[:, 'text_clean'], 35, 15, ['nmf']) # 2004 - 2008

NMF
Topic 0:
развитие сотрудничество международный экономический работа новый важный решение организация задача регион являться интерес становиться проблема
Topic 1:
оон резолюция безопасность совет сб право международный устав генеральный член организация правозащитный урегулирование секретарь сессия
Topic 2:
сотрудничество вопрос обсуждать визит переговоры двусторонний президент договариваться договоренность торговый-экономический встреча ход сфера сегодня взаимодействие
Topic 3:
обсе организация бдипча смид выборы безопасность государство-участник реформа корзина мониторинг наблюдение наблюдатель бюро председательство обязательство
Topic 4:
политика мир внешний международный мировой война холодный глобальный современный интерес дипломатия коллективный новый угроза цивилизация
Topic 5:
палестинский израиль палестинец пна израильский аббас квартет дорожный карта территория палестино-израильский хамас урегулирование посредник ближневосточный
Topic 6:
соотечественник русский язык рубеж 

In [260]:
get_topics(sorted_dfs2[2].loc[:, 'text_clean'], 38, 15, ['nmf']) # 2008 - 2014

NMF
Topic 0:
международный развитие сотрудничество новый работа мир интерес сегодня глобальный проблема важный основа мировой задача решение
Topic 1:
сирия сирийский оппозиция сара вооруженный дамаск женевский аннан насилие правительство прекращение лаг сириец боевик химический
Topic 2:
визит сотрудничество переговоры двусторонний федерация обсуждать республика вопрос состояться президент ход взаимодействие встреча рабочий актуальный
Topic 3:
осетия грузия южный абхазия грузинский тбилиси республика саакашвили закавказье август граница сила абхазский югоосетинский неприменение
Topic 4:
соотечественник рубеж конференция всемирный зарубежный конгресс проживать правительственный координационный федерация община диаспора комиссия работа русский
Topic 5:
оон резолюция сб совет безопасность генеральный международный решение организация ассамблея миссия сессия генассамблея генсекретарь секретарь
Topic 6:
гражданин сша американский посольство ребенок консульский суд власть информация сотрудник

In [261]:
get_topics(sorted_dfs2[3].loc[:, 'text_clean'], 65, 15, ['nmf']) # С 2014

NMF
Topic 0:
международный интерес мир проблема безопасность регион усилие решение работа развитие основа процесс принцип диалог новый
Topic 1:
сша американский вашингтон санкция администрация американец госдепартамент против санкционный российский-американский давление обама госсекретарь вводить военный
Topic 2:
сирия сирийский оппозиция урегулирование процесс сириец межсирийский женева правительство террорист сара политический переговоры астанинский группа
Topic 3:
украина украинский киев киевский власть юго-восток действие заявление нынешний язык верховный одесса очередной порошенко кризис
Topic 4:
визит актуальный переговоры двусторонний повестка рабочий вопрос обсуждать мнение региональный международный республика федерация обмен день
Topic 5:
война победа великий память мировой второй отечественный история летие нацизм ветеран исторический народ нацистский мероприятие
Topic 6:
химический озхо оружие применение запрещение химоружие вещество кзхо сирия конвенция отравлять муфс докл